In [1]:
import os
import sys
sys.path.insert(0, '../funzioni')
from funzioni import *
import json
import os
import sys
import re
import pandas as pd
sys.path.append('../../')

In [2]:
run_dataq()

Questa la documentazione sulla suite: Main 
 http://localhost:9000/view/great_expectations/uncommitted/data_docs/local_site/expectations/Main.html


Calculating Metrics:   0%|          | 0/189 [00:00<?, ?it/s]

Consulta i risultati del data_asset_name "scraping_all_20230216" al link:
http://localhost:9000/view/great_expectations/uncommitted/data_docs/local_site/validations/Main/20230226-220618-scraping_all_20230216/20230226T220618.926535Z/c4e91d3d0445ea05aa5a84cca138ff6f.html


Calculating Metrics:   0%|          | 0/189 [00:00<?, ?it/s]

Consulta i risultati del data_asset_name "scraping_all_20230217" al link:
http://localhost:9000/view/great_expectations/uncommitted/data_docs/local_site/validations/Main/20230226-220622-scraping_all_20230217/20230226T220622.025063Z/3efa08eb01bba7de1266fd8bb26c5cf3.html


...

Consulta i risultati del data_asset_name "scraping_all_20230219" al link:
http://localhost:9000/view/great_expectations/uncommitted/data_docs/local_site/validations/Main/20230226-220625-scraping_all_20230219/20230226T220625.344867Z/e9bd252c14835612b1e4ae4a3ac712c0.html


Calculating Metrics:   0%|          | 0/189 [00:00<?, ?it/s]

Consulta i risultati del data_asset_name "scraping_all_20230224" al link:
http://localhost:9000/view/great_expectations/uncommitted/data_docs/local_site/validations/Main/20230226-220628-scraping_all_20230224/20230226T220628.614688Z/4d7cdc08e0ee331cce276f99b1b7442b.html


Calculating Metrics:   0%|          | 0/189 [00:00<?, ?it/s]

Consulta i risultati del data_asset_name "scraping_all_20230225" al link:
http://localhost:9000/view/great_expectations/uncommitted/data_docs/local_site/validations/Main/20230226-220632-scraping_all_20230225/20230226T220632.386092Z/44443cc4de44ba37642b43bd85f3a657.html
Checkpoint aggiornati o aggiunti:
 ['checkpoint_20230216', 'checkpoint_20230217', 'checkpoint_20230219', 'checkpoint_20230224', 'checkpoint_20230225', 'main_checkpoint']


In [ ]:
df  = parse_json()
df

In [ ]:
df_scrapingdate = df.iloc[:,:-3].groupby('scraping_date').agg({'element_count':max,'unexpected_percent':'mean', 'unexpected_percent_nonmissing':'mean', 'missing_percent':'mean' })
df_scrapingdate

In [ ]:
df_scrapingdate_column = df.iloc[:,:-3].groupby(['scraping_date', 'column']).agg({'element_count':max,'unexpected_percent':'mean', 'unexpected_percent_nonmissing':'mean', 'missing_percent':'mean' }).reset_index()
df_scrapingdate_column

In [ ]:
df_column = df.iloc[:,:-3].groupby(['column']).agg({'element_count':'mean','unexpected_percent':'mean', 'unexpected_percent_nonmissing':'mean', 'missing_percent':'mean' }).reset_index()
df_column

In [ ]:
df_expectation = df.iloc[:,:-3].groupby(['expectation_type']).agg({'element_count':max,'unexpected_percent':'mean', 'unexpected_percent_nonmissing':'mean', 'missing_percent':'mean' }).reset_index()
df_expectation